# Import dacl10k to fiftyone

For the install of fiftyone, please refer to: https://docs.voxel51.com/getting_started/install.html.
Here's a thumbnail, showing fiftyone after you have run the following two cells:


<img src="../assets/fiftyone_thumbnail.png" alt="Fiftyone thumbnail" width="600"/>


In [1]:
import json
import fiftyone as fo
from pathlib import Path
import numpy as np
from datetime import date, datetime

In [2]:
# Create samples for your data
images_patt = Path("../../../data/dacl10k_dacl-challenge/images/train") # Path to your images!
annotations_patt = Path("../../../data/dacl10k_dacl-challenge/annotations/train") # Path to your annotations!

samples = []
for annot_path in annotations_patt.iterdir():
    if annot_path.is_file() and annot_path.suffix == ".json":    
        with open(annot_path, "r") as f:
            annot = json.load(f)
    
    img_path_full_str = str(images_patt / (str(annot_path.stem) + ".jpg"))
    tag = annot['split']

    # Metadata:
    metadata = fo.ImageMetadata.build_for(img_path_full_str)
    sample = fo.Sample(filepath=img_path_full_str, metadata=metadata, tags=[tag],
                       created_at=datetime.utcnow(), created_date=date.today(), name=annot_path.stem)

    # All polygon dependant info:
    detections = []
    segmentations = []
    classifications_set = set()
    for shape in annot["shapes"]:
        label = shape["label"]
        points = shape["points"]
        try:
            poly_points = [[(x/annot['imageWidth'],y/annot['imageHeight']) for x,y in points]]
        except ValueError as e:
            print("ValueError: ", annot_path.stem)
            print(e)
            continue

        # Multi-label:
        classifications_set.add(label)   
        
        # Bounding box:
        top_left = np.amin(points, 0)
        bot_right = np.amax(points,0)
        top_left = [top_left[0], 
                    top_left[1]]
        bot_right = [(bot_right[0]),
                     (bot_right[1])]
        width = bot_right[0]-top_left[0]
        height = bot_right[1]-top_left[1]
        bbox = [top_left[0]/annot['imageWidth'], top_left[1]/annot['imageHeight'], 
                width/annot['imageWidth'], height/annot['imageHeight']] # [top-left-x, top-left-y, width, height]
        bbox_ = [i.tolist() for i in bbox]
        detections.append(
            fo.Detection(label=label, bounding_box=bbox, iscrowd=False)
        )
        segmentations.append(
            fo.Polyline(label=label, points=poly_points, closed=True,filled=True, iscrowd=False)
        )

    # Store detections in a field name of your choice
    sample["semseg_ground_truth_filled"] = fo.Polylines(polylines=segmentations)
    sample["od_ground_truth"] = fo.Detections(detections=detections)

    classifications_list = [fo.Classification(label=label) for label in classifications_set]
    sample["multi_ground_truth"] = fo.Classifications(classifications=classifications_list)
    
    samples.append(sample)

# Create dataset
dataset = fo.Dataset("dacl10k_train")
dataset.add_samples(samples)
session = fo.launch_app(dataset)

 100% |███████████████| 6935/6935 [52.1s elapsed, 0s remaining, 141.0 samples/s]      
